# Movie recommendation on Amazon SageMaker with Factorization Machines

### Background

In many ways, recommender systems were a catalyst for the current popularity of machine learning.  One of Amazon's earliest successes was the "Customers who bought this, also bought..." feature, while the million dollar Netflix Prize spurred research, raised public awareness, and inspired numerous other data science competitions.

Recommender systems can utilize a multitude of data sources and ML algorithms, and most combine various unsupervised, supervised, and reinforcement learning techniques into a holistic framework.  However, the core component is almost always a model which which predicts a user's rating (or purchase) for a certain item based on that user's historical ratings of similar items as well as the behavior of other similar users.

Factorization Machines have shown themselves to be very effective at this task, and Amazon SageMaker includes an almost perfectly scalable implementation of a distributed Factorization Machine algorithm.  Today we'll use that to build a recommender system.  We'll start by bringing in and preparing our dataset, and then discuss the algorithm and training process, and finish by deploying to a real-time endpoint for generating predictions.

### Imports

Let's start by importing our necessary modules here, to get it out of the way.  Also, update the s3 bucket, prefix, or IAM role as needed.

In [51]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker.predictor import json_deserializer

import boto3, csv, io, json
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Defaults

We will set a project name used to group Amazon SageMaker and Amazon S3 resources related to this specific notebook

In [2]:
project_name = 'sagemaker-workshop-movie-recommender'
train_instance_count = 1
train_instance_type = "ml.c5.18xlarge"
train_dataset_uri = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip' 

### Helper Function: Identify SageMaker IAM service role

SageMaker resources will need access to several resources to complete training jobs

In [3]:
def get_sagemaker_role():
    try:
        # if running within Amazon SageMaker Notebook Instance, get current execution role
        role = get_execution_role()
    except:
        # otherwise, guess the role
        result = %sx aws iam list-roles | grep -e 'role/service-role' | grep -i 'sagemaker' | tr -d ' '
        role = result[0][7:-2]
        print("Found potential SageMaker service role: {}".format(role))
    return role

In [4]:
role = get_sagemaker_role()

Found potential SageMaker service role: arn:aws:iam::306280812807:role/service-role/AmazonSageMaker-ExecutionRole-20180117T091311


### Helper Function: Identify SageMaker Container for specified region

Set boto3 clients to use a valid SageMaker region and to select the proper container to use for training and endpoint deployment based on the specific algorithm to be used

In [5]:
available_regions=['us-east-1','us-east-2','us-west-2','eu-west-1'] # available Amazon SageMaker regions
region_name = boto3.Session().region_name
if region_name not in available_regions:
    region_name=available_regions[0]
print('Setting SageMaker region to: {}'.format(region_name))
default_bucket = '{}-{}'.format(project_name, region_name)
print('Setting default S3 bucket to: {}'.format(default_bucket))

Setting SageMaker region to: us-east-1
Setting default S3 bucket to: sagemaker-workshop-movie-recommender-us-east-1


## Helper Function: Protocol Buffer formatting

In [6]:
prefix = 'protobuf_5'
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)

In [7]:
### TODO: If not exists, create S3 bucket
s3 = boto3.client("s3", region_name=region_name)
## check if bucket exists
## if not...
s3.create_bucket(Bucket=default_bucket)

{'Location': '/sagemaker-workshop-movie-recommender-us-east-1',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Thu, 03 May 2018 12:24:16 GMT',
   'location': '/sagemaker-workshop-movie-recommender-us-east-1',
   'server': 'AmazonS3',
   'x-amz-id-2': 'dGLcTNEL1k7wO6+mRkBtE5kHZ5OfVuBmejhKtKiJiUxVHBFU6ALwvVbyfsM/S1v4bZh8SVofb2c=',
   'x-amz-request-id': 'B05E272E28C91624'},
  'HTTPStatusCode': 200,
  'HostId': 'dGLcTNEL1k7wO6+mRkBtE5kHZ5OfVuBmejhKtKiJiUxVHBFU6ALwvVbyfsM/S1v4bZh8SVofb2c=',
  'RequestId': 'B05E272E28C91624',
  'RetryAttempts': 0}}

### Download and look at our data
Let's get our data. Factorization Machines work well as recommendation solutions, such as movie recommendations! Also, FM's do great on lots of data, so let's get something heavy. Maybe the MovieLens dataset, with 20 million ratings?

In [49]:
!rm -f dataset.zip
!wget -O /tmp/dataset.zip $train_dataset_uri
!unzip -j -o /tmp/dataset.zip -d /tmp/dataset

--2018-05-05 04:43:36--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.34.235
Connecting to files.grouplens.org (files.grouplens.org)|128.101.34.235|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248434223 (237M) [application/zip]
Saving to: ‘/tmp/dataset.zip’

/tmp/dataset.zip    100%[===================>] 236.92M  56.0MB/s    in 4.4s    

2018-05-05 04:43:40 (54.0 MB/s) - ‘/tmp/dataset.zip’ saved [248434223/248434223]

Archive:  /tmp/dataset.zip
  inflating: /tmp/dataset/genome-scores.csv  
  inflating: /tmp/dataset/genome-tags.csv  
  inflating: /tmp/dataset/links.csv  
  inflating: /tmp/dataset/movies.csv  
  inflating: /tmp/dataset/ratings.csv  
  inflating: /tmp/dataset/README.txt  
  inflating: /tmp/dataset/tags.csv   


Let's take a look at the top of this file, shall we? This should show us the range of user ids, which we will need later.

In [9]:
%cd /tmp/dataset
!head -5000000 ratings.csv > ratings.trunc.csv
!mv ratings.trunc.csv ratings.csv
!head -10 ratings.csv
!tail -10 ratings.csv
result=!tail -1 ratings.csv | cut -d',' -f1
user_count=int(result[0])
print("Users: {}".format(user_count))

/tmp/dataset
userId,movieId,rating,timestamp
1,110,1.0,1425941529
1,147,4.5,1425942435
1,858,5.0,1425941523
1,1221,5.0,1425941546
1,1246,5.0,1425941556
1,1968,4.0,1425942148
1,2762,4.5,1425941300
1,2918,5.0,1425941593
1,2959,4.0,1425941601
51433,172,5.0,1394863719
51433,173,5.0,1394863483
51433,208,5.0,1394863612
51433,333,5.0,1394863271
51433,442,5.0,1394863961
51433,541,5.0,1394863673
51433,661,5.0,1394863273
51433,783,4.0,1394863276
51433,1035,0.5,1394863180
51433,1320,5.0,1394863183
Users: 51433


### Prepare our data

We remove the header, then shuffle the data before splitting it into training and prediction segments.

In [10]:
!tail -n +2 ratings.csv | shuf -o ratings.shuffled

result=!wc -l ratings.shuffled | cut -d' ' -f1
lines_total=result[0]
lines_split=int(int(lines_total)/10)
!split --lines $lines_split ratings.shuffled

!cat xa{a..h} > ratings.shuffled.training
!cat xai > ratings.shuffled.validation
!cat xaj > ratings.shuffled.prediction

result=!wc -l ratings.shuffled.training | cut -d' ' -f1
lines_training=int(result[0])
print("Training set: {}".format(lines_training))
!head -n5 ratings.shuffled.training

result=!wc -l ratings.shuffled.validation | cut -d' ' -f1      
lines_validation=int(result[0])
print("Validation set: {}".format(lines_validation))  
!head -n5 ratings.shuffled.validation
      
result=!wc -l ratings.shuffled.prediction | cut -d' ' -f1
lines_prediction=int(result[0])
print("Prediction set: {}".format(lines_prediction))
!head -n5 ratings.shuffled.prediction


Training set: 3999992
50100,34437,4.5,1213523382
25930,6281,3.5,1197551064
1757,282,3.0,953069447
2022,5452,3.5,1345325819
30432,555,2.0,832423645
Validation set: 499999
13222,2761,4.0,1111442322
5953,1246,5.0,1432706322
32984,46083,2.5,1282142417
24528,1597,2.0,1217279586
37791,58494,3.5,1219669123
Prediction set: 499999
48309,2402,0.5,1172106649
31104,45672,2.5,1466591524
2359,2496,3.0,1203249423
36379,223,5.0,1003617270
7386,7802,4.0,1226628462


### Build training set and test set

When using Factorization Machines for building a recommender system, the algorithm expects the data to look something like:

|Rating|User1|User2|...|UserN|Movie1|Movie2|Movie3|...|MovieM|
|---|---|---|---|---|---|---|---|---|---|
|4|1|0|...|0|1|0|0|...|0|
|5|1|0|...|0|0|1|0|...|0|
|3|0|1|...|0|1|0|0|...|0|
|4|0|1|...|0|0|0|1|...|0|

- Our target column is the rating for that user movie combination.  This could be the number of stars given, or could be a binarized version (movies with ratings 4 and above are set to `1`, otherwise `0`).  
- We have a set of `N` features which are a one-hot encoding of user.  They only take a value of `1` to indicate observations which belong to that customer.
- We also have a set of `M` features which are a one-hot encoding of movie.

Sagemaker's Factorization Machines can handle sparse data, meaning we only store the non-zero entries, but to contruct the sparse feature matrices we need to figure out the size of the training and testing sets. We also normalized the movie ids here.

In [11]:
normalizedMovieIdDict = {}
normalizedIdMovies = 0

with open('ratings.shuffled','r') as f:
    samples = csv.reader(f,delimiter=',')
    for userId,movieId,rating,timestamp in samples:
        if movieId not in normalizedMovieIdDict:
            normalizedMovieIdDict[movieId] = normalizedIdMovies
            normalizedIdMovies = normalizedIdMovies + 1


nbUsers = user_count # Our max user ID in the tail of the user sorted file
nbMovies = normalizedIdMovies

nbFeatures = nbUsers+nbMovies

Now, we can read our training and test datsets in.

In [12]:
def loadDataset(filename, total_lines, columns):
    # Features are one-hot encoded in a sparse matrix
    
    X = lil_matrix((total_lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line = 0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter=',')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            normalizedMovieId = normalizedMovieIdDict[movieId]
            X[line,int(nbUsers)+int(normalizedMovieId)-1] = 1
            if float(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line = line+1
            
    Y = np.array(Y).astype('float32') 
    return X,Y

In [13]:
%%time

X_train, Y_train = loadDataset('ratings.shuffled.training', lines_training, nbFeatures)
X_test, Y_test = loadDataset('ratings.shuffled.validation', lines_validation, nbFeatures)

CPU times: user 1min 17s, sys: 876 ms, total: 1min 17s
Wall time: 1min 17s


### Convert to protobuf and save to S3
We train our models from data in S3; writing a helper function here to help us with that.

In [14]:
%%time

train_data_location = writeDatasetToProtobuf(X_train, Y_train, default_bucket, train_prefix, train_key)    
test_data_location  = writeDatasetToProtobuf(X_test, Y_test, default_bucket, test_prefix, test_key)

train_data_location = "s3://{}/{}/train/{}".format(default_bucket, prefix, train_key)  
test_data_location  = "s3://{}/{}/test/{}".format(default_bucket, prefix, test_key)
  
print(train_data_location)
print(test_data_location)

s3://sagemaker-workshop-movie-recommender-us-east-1/protobuf_5/train/train.protobuf
s3://sagemaker-workshop-movie-recommender-us-east-1/protobuf_5/test/test.protobuf
CPU times: user 9min 17s, sys: 6.79 s, total: 9min 24s
Wall time: 9min 6s


### Run Factorization Machine training job with validation
Sagemaker algorithms are stored as Docker containers in ECS, and we need the URI of the containers. This will be region specific.

In [15]:
fm_containers = {
    'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
    'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
    'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
    'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

training_image = fm_containers[region_name]

Training time! Some things to note here that might not be obvious:
<ul>
    <li>You can specify the hardware you want to use; Sagemaker will create your cluster and deploy your image</li>
    <li>You can optionally proviode a test dataset as well, which we have done here</li>
    <li>The metrics will be printed out after each epoch and mini batch</li>
</ul>

In order to train our Factorization Machine, we'll need to specify some hyperparameters.  First, let's think about what a Factorization Machine does:  

$$\hat{r} = w_0 + \sum_{i} {w_i x_i} + \sum_{i} {\sum_{j > i} {\langle v_i, v_j \rangle x_i x_j}}$$

where $\hat{r}$ is the movie rating, $x_i$ are the one-hot encoded user and movie indicators, $w_i$ are linear weights, and the second term represents the pairwise feature interactions as a reduced dimension factorization.  This reduction in dimensionality allows us to find a smaller, more generalizable representation of information in the large sparse input dataset.  Key hyperparameters are therefore:

- `feature_dim`: How big our initial movie 
- `num_factors`: How large our reduced dimensionality representation of interactions should be

We also specify hyperparameters like `mini_batch_size` and `epochs` which can be tuned for improved performance.

Hit "play" and sit back to let Sagemaker do all the work here

In [16]:
%%time

fm_trained_model_location  = 's3://{}/{}/{}/output'.format(default_bucket, prefix, "fm")

fm = sagemaker.estimator.Estimator(
    training_image,
    role, 
    train_instance_count=train_instance_count, 
    train_instance_type=train_instance_type,
    output_path=fm_trained_model_location,
    sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(
    feature_dim=nbFeatures,
    predictor_type='binary_classifier',
    mini_batch_size=1000,
    num_factors=64,
    epochs=3)

fm.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: factorization-machines-2018-05-03-12-35-11-657


.............................................
Docker entrypoint called with argument(s): train
[05/03/2018 12:38:50 INFO 140437905205056] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_init_method': u'normal', u'linear_init_method': u'normal', u'linear_lr': u'0.001', u'factors_init_method': u'normal', u'_tuning_objective_metric': u'', u'bias_wd': u'0.01', u'use_linear': u'true', u'_speedometer_period': u'500', u'bias_lr': u'0.1', u'mini_batch_size': u'1000', u'_use_full_symbolic': u'true', u'bias_init_sigma': u'0.01', u'_num_gpus': u'auto', u'_data_format': u'record', u'factors_wd': u'0.00001', u'linear_wd': u'0.001', u'_kvstore': u'auto', u'_learning_rate': u'1.0', u'_optimizer': u'adam'}
[05/03/2018 12:38:50 INFO 14043

[05/03/2018 12:39:26 INFO 140437905205056] Epoch[1] Batch [2500]#011Speed: 180327.76 samples/sec#011binary_classification_accuracy=0.719085#011binary_classification_cross_entropy=0.618783#011binary_f_1.000=0.713168
[05/03/2018 12:39:29 INFO 140437905205056] Epoch[1] Batch [3000]#011Speed: 182907.59 samples/sec#011binary_classification_accuracy=0.719773#011binary_classification_cross_entropy=0.617214#011binary_f_1.000=0.713889
[05/03/2018 12:39:32 INFO 140437905205056] Epoch[1] Batch [3500]#011Speed: 181952.50 samples/sec#011binary_classification_accuracy=0.720278#011binary_classification_cross_entropy=0.615718#011binary_f_1.000=0.713891
#metrics {"Metrics": {"update.time": {"count": 1, "max": 21953.569889068604, "sum": 21953.569889068604, "min": 21953.569889068604}}, "EndTime": 1525351174.752013, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1525351152.797685}

[05/03/2018 12:39:34 INFO 140437905205056] #progress_metric: 

### Deploy model
Now our trained model is stored, and you can actually see this in the Sagmaker console. We can go ahead and deploy this.

In [24]:
%%time

fm_predictor = fm.deploy(instance_type='ml.c5.2xlarge', initial_instance_count=1)

INFO:sagemaker:Creating model with name: factorization-machines-2018-05-03-12-56-01-209
INFO:sagemaker:Creating endpoint with name factorization-machines-2018-05-03-12-35-11-657


--------------------------------------------------------------!CPU times: user 324 ms, sys: 8 ms, total: 332 ms
Wall time: 5min 15s


We are going to make requests to our endpoint for prediction. Our data is really sparse (only 2 values in each vector row of 165237), and it doesn't make sense to send matrices that are mostly empty. To solve this problem, we are going to go ahead and create a protobuf serializer to send content in a sparse format is supported by sagemaker.

You can see more formats here: https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-inference.html

In [25]:
def sparse_protobuf_serializer(data):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, data)
    buf.seek(0)
    return buf

fm_predictor.content_type = 'application/x-recordio-protobuf'
fm_predictor.serializer = sparse_protobuf_serializer
fm_predictor.deserializer = json_deserializer

### Run predictions
We made a dataset to run prediction earlier in this notebook. Let's go ahead and load that.

In [26]:
X_pred, Y_pred = loadDataset('ratings.shuffled.prediction', lines_prediction , nbFeatures)

Let's grab single row and run prediction on it.

In [27]:
fm_result = fm_predictor.predict(X_pred[:1])
print(fm_result)
true_labels = Y_pred[:1]

for l,r in zip(true_labels, fm_result['predictions']):
    print("TrueLabel:{} PredictedLabel:{} Score:{}".format(l, int(r['predicted_label']), r['score']))

{'predictions': [{'score': 0.4593372642993927, 'predicted_label': 0.0}]}
TrueLabel:0.0 PredictedLabel:0 Score:0.4593372642993927


## Validation of training job's accuracy
Now that we've sanity checked that, let's just run inference on the entire prediction dataset of 2 million rows. Our feature dimension is wide but our data is mostly sparse, so we should be able to do relatively big batches of inferences. (This will take a couple of minutes)

In [32]:
%%time

batch_size = lines_prediction
predictions = []

batches = int(X_pred.shape[0]/batch_size)

for i in range(batches):
    X_sample = X_pred[ (i*batch_size) : ((i+1)*batch_size) ]
    result = fm_predictor.predict(X_sample)
    predictions += [r['predicted_label'] for r in result['predictions']]

predictions = np.array(predictions)

CPU times: user 45.2 s, sys: 596 ms, total: 45.8 s
Wall time: 49 s


In [45]:
fm_result = fm_predictor.predict(X_pred[1000:1003])
print(fm_result)
print(Y_pred[1000:1003])

{'predictions': [{'score': 0.7121307253837585, 'predicted_label': 1.0}, {'score': 0.18819570541381836, 'predicted_label': 0.0}, {'score': 0.5745882987976074, 'predicted_label': 1.0}]}
[1. 0. 1.]


Let's see what percentage we recommended correctly. This should be roughly equivalent to our testing metric that was reported during training, but provides independent validation.

In [34]:
predictions[0]
np.sum(Y_pred == predictions) / len(Y_pred)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


0.0

# This is a substantial improvement over a naive baseline ... add more here

## Clean up, and finish!
And we are done!

In [23]:
#fm.delete_endpoint()

INFO:sagemaker:Deleting endpoint with name: factorization-machines-2018-05-03-12-35-11-657
